# Introduction of DataFrame and Spark SQL


##### Reading the file from DataFrame 

In [1]:
#Import the modules

from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [2]:
#Create session in order to be capable of accessing the all Spark API
spark = SparkSession.builder.appName('Basics of Spark Dataframe').config('spark.some.config.value','some-value').getOrCreate()

In [3]:
#Define data schema for file we want to read
PurchaseSchema = StructType ([
    StructField("Date",DateType(),True),
    StructField("Time",StringType(),True),
    StructField("City",StringType(),True),
    StructField("Item",StringType(),True),
    StructField("Total_Amount",FloatType(),True),
    StructField("Payment",StringType(),True),
    ])


In [4]:
#Read csv file with our defined schema into spark dataframe, and use 'TAB' as delimiter
PurchaseDataFrame = spark.read.csv("dataset/purchases.csv",header=True,schema=PurchaseSchema,sep='\t')

#Display 3 rows of Dataframe
PurchaseDataFrame.show(5)

+----------+-----+----------+-------------------+------------+----------+
|      Date| Time|      City|               Item|Total_Amount|   Payment|
+----------+-----+----------+-------------------+------------+----------+
|2012-01-01|09:00|  San Jose|     Men's Clothing|      214.05|      Amex|
|2012-01-01|09:00|Fort Worth|   Women's Clothing|      153.57|      Visa|
|2012-01-01|09:00| San Diego|              Music|       66.08|      Cash|
|2012-01-01|09:00|Pittsburgh|       Pet Supplies|      493.51|  Discover|
|2012-01-01|09:00|     Omaha|Children's Clothing|      235.63|MasterCard|
+----------+-----+----------+-------------------+------------+----------+
only showing top 5 rows



In [5]:
#Count no.of rows and print the dataframe schema and print statistics of input data

num_rows = PurchaseDataFrame.count()
print ('Number of rows:', num_rows)

Number of rows: 4138476


In [6]:
#Display the statistics of particular column data 
PurchaseDataFrame.describe('Total_Amount').show()

+-------+------------------+
|summary|      Total_Amount|
+-------+------------------+
|  count|           4138476|
|   mean|249.96108549398525|
| stddev| 144.3174111542959|
|    min|               0.0|
|    max|            499.99|
+-------+------------------+



In [7]:
#Display the dataframe schema
PurchaseDataFrame.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Total_Amount: float (nullable = true)
 |-- Payment: string (nullable = true)



# Creating a new dataframe from an existing dataframe

In [8]:
#Create new dataframe for two colums
newDataFrame = PurchaseDataFrame.select(PurchaseDataFrame['City'],PurchaseDataFrame['Total_Amount'])

newDataFrame.show(3)


+----------+------------+
|      City|Total_Amount|
+----------+------------+
|  San Jose|      214.05|
|Fort Worth|      153.57|
| San Diego|       66.08|
+----------+------------+
only showing top 3 rows



In [9]:
newDataFrame.printSchema()

root
 |-- City: string (nullable = true)
 |-- Total_Amount: float (nullable = true)



In [10]:
#Adding an constant value to particular column

PurchaseDataFrame.select(PurchaseDataFrame['City'],PurchaseDataFrame['Total_Amount']+30).show(3)

+----------+-------------------+
|      City|(Total_Amount + 30)|
+----------+-------------------+
|  San Jose|             244.05|
|Fort Worth|             183.57|
| San Diego|              96.08|
+----------+-------------------+
only showing top 3 rows



In [11]:
#Filter the dataframe using defined condition

PurchaseDataFrame.filter(PurchaseDataFrame['Total_Amount'] > 200).show(5)


+----------+-----+----------+-------------------+------------+----------+
|      Date| Time|      City|               Item|Total_Amount|   Payment|
+----------+-----+----------+-------------------+------------+----------+
|2012-01-01|09:00|  San Jose|     Men's Clothing|      214.05|      Amex|
|2012-01-01|09:00|Pittsburgh|       Pet Supplies|      493.51|  Discover|
|2012-01-01|09:00|     Omaha|Children's Clothing|      235.63|MasterCard|
|2012-01-01|09:00|  Stockton|     Men's Clothing|      247.18|MasterCard|
|2012-01-01|09:00|    Austin|            Cameras|       379.6|      Visa|
+----------+-----+----------+-------------------+------------+----------+
only showing top 5 rows



In [12]:
#Sorting DataFrame by certain column:
sortByCity = PurchaseDataFrame.orderBy('City').show(5)

+----------+-----+-----------+----------------+------------+----------+
|      Date| Time|       City|            Item|Total_Amount|   Payment|
+----------+-----+-----------+----------------+------------+----------+
|2012-10-07|11:11|Albuquerque|    Pet Supplies|       308.7|      Visa|
|2012-10-07|11:41|Albuquerque|           Music|      365.64|      Visa|
|2012-10-07|11:13|Albuquerque|Women's Clothing|      419.49|  Discover|
|2012-10-07|10:39|Albuquerque|    Pet Supplies|       401.3|MasterCard|
|2012-10-07|11:18|Albuquerque|          Crafts|      475.77|      Visa|
+----------+-----+-----------+----------------+------------+----------+
only showing top 5 rows



In [13]:
#Calculating number of txn in each city
numtxneachCity = PurchaseDataFrame.groupBy('City').count()
numtxneachCity.show(5)

+---------------+-----+
|           City|count|
+---------------+-----+
|North Las Vegas|40013|
|        Phoenix|40333|
|          Omaha|40209|
|      Anchorage|39806|
|        Anaheim|40086|
+---------------+-----+
only showing top 5 rows



# Indexing and Accessing DataFrame

Spark dataframe is distributed into clusters we cant access it by ROW,COLUMNS as in pandas dataframe for  example. There is an alternative wat to do that by creating new column with 'incremental ID'. Then we can access by row using .filter() function to the incremental ID column.

In [14]:
#import monotonically_increasing_id

from pyspark.sql.functions import monotonically_increasing_id

In [15]:
newPurchaseDataFrame = PurchaseDataFrame.withColumn("Index",monotonically_increasing_id())

In [16]:
newPurchaseDataFrame.show(5)

+----------+-----+----------+-------------------+------------+----------+-----+
|      Date| Time|      City|               Item|Total_Amount|   Payment|Index|
+----------+-----+----------+-------------------+------------+----------+-----+
|2012-01-01|09:00|  San Jose|     Men's Clothing|      214.05|      Amex|    0|
|2012-01-01|09:00|Fort Worth|   Women's Clothing|      153.57|      Visa|    1|
|2012-01-01|09:00| San Diego|              Music|       66.08|      Cash|    2|
|2012-01-01|09:00|Pittsburgh|       Pet Supplies|      493.51|  Discover|    3|
|2012-01-01|09:00|     Omaha|Children's Clothing|      235.63|MasterCard|    4|
+----------+-----+----------+-------------------+------------+----------+-----+
only showing top 5 rows



In [17]:
row2To4 = newPurchaseDataFrame.filter((newPurchaseDataFrame['index']>=2) & (newPurchaseDataFrame['index']<=4))
row2To4.show()

+----------+-----+----------+-------------------+------------+----------+-----+
|      Date| Time|      City|               Item|Total_Amount|   Payment|Index|
+----------+-----+----------+-------------------+------------+----------+-----+
|2012-01-01|09:00| San Diego|              Music|       66.08|      Cash|    2|
|2012-01-01|09:00|Pittsburgh|       Pet Supplies|      493.51|  Discover|    3|
|2012-01-01|09:00|     Omaha|Children's Clothing|      235.63|MasterCard|    4|
+----------+-----+----------+-------------------+------------+----------+-----+



*to access it by row and columns use .select() function*

In [18]:
dataRow2ColumnTotal_Amount = newPurchaseDataFrame.filter(newPurchaseDataFrame['index'] ==2).select("Total_Amount").show()

+------------+
|Total_Amount|
+------------+
|       66.08|
+------------+



# Using SQL query in DataFrame

Creating a new dataframe from the subset from existing dataframe


In [19]:
#We need to make SQL temp view for our dataframe

PurchaseDataFrame.createOrReplaceTempView("PurchaseSQL")

In [20]:
#select Total_Amount and Payment columns from SQL temp view

anotherDataFrame = spark.sql("Select Total_Amount,Payment,City from purchaseSQL")

anotherDataFrame.show(5)

+------------+----------+----------+
|Total_Amount|   Payment|      City|
+------------+----------+----------+
|      214.05|      Amex|  San Jose|
|      153.57|      Visa|Fort Worth|
|       66.08|      Cash| San Diego|
|      493.51|  Discover|Pittsburgh|
|      235.63|MasterCard|     Omaha|
+------------+----------+----------+
only showing top 5 rows



# Sorting data by certain columns

In [21]:
# sorting data by City column
orderByCity = spark.sql("select * from purchaseSQL order by City")

orderByCity.show(5)


+----------+-----+-----------+----------------+------------+----------+
|      Date| Time|       City|            Item|Total_Amount|   Payment|
+----------+-----+-----------+----------------+------------+----------+
|2012-10-07|11:11|Albuquerque|    Pet Supplies|       308.7|      Visa|
|2012-10-07|11:41|Albuquerque|           Music|      365.64|      Visa|
|2012-10-07|11:13|Albuquerque|Women's Clothing|      419.49|  Discover|
|2012-10-07|10:39|Albuquerque|    Pet Supplies|       401.3|MasterCard|
|2012-10-07|11:18|Albuquerque|          Crafts|      475.77|      Visa|
+----------+-----+-----------+----------------+------------+----------+
only showing top 5 rows



In [22]:
#SQL with Where conditions

filterAndSortWithSQL = spark.sql("select * from purchaseSQL where Total_Amount > 200 order by city")

filterAndSortWithSQL.show(5)

+----------+-----+-----------+-----------------+------------+----------+
|      Date| Time|       City|             Item|Total_Amount|   Payment|
+----------+-----+-----------+-----------------+------------+----------+
|2012-10-07|11:18|Albuquerque|           Crafts|      475.77|      Visa|
|2012-10-07|11:54|Albuquerque|Health and Beauty|       480.4|MasterCard|
|2012-10-07|11:40|Albuquerque|             Toys|      299.63|MasterCard|
|2012-10-07|10:42|Albuquerque|   Sporting Goods|      351.13|      Cash|
|2012-10-07|11:41|Albuquerque|            Music|      365.64|      Visa|
+----------+-----+-----------+-----------------+------------+----------+
only showing top 5 rows

